In [ ]:
# Assignment #3 Vysotskiy Roman J4132c

In [ ]:
"""
Create spark session with 2 cores and 2g memory on a driver (you will work locally)
"""

'\nCreate spark session with 2 cores and 2g memory on a driver (you will work locally)\n'

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("taskname").config("spark.sql.repl.eagerEval.enabled", True).getOrCreate()
# spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [ ]:
"""
Using Spark UI link below you can see tasks submitted to spark
Also you can see cached values
"""

In [3]:
spark

In [4]:
import random

from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, udf, monotonically_increasing_id, mean

In [ ]:
"""
Read dataset
"""

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data = spark.read.json('/content/drive/MyDrive/cards_(2).json') #('/content/cards_(2).json')
data.show(10)

+----------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+--------------------+-------+--------------------+------+---------+---------+---------------+---------+--------------------+---------------+-------------------+--------+--------------------+-----------+----+---------+--------------+-------+-----------+------------------+--------------------+-----------+
|          artist|attack|cardClass|classes|collectible|collectionText|cost|dbfId|durability|elite|           entourage|faction|              flavor|health|hideStats|howToEarn|howToEarnGolden|       id|           mechanics|multiClassGroup|               name|overload|    playRequirements|playerClass|race|   rarity|referencedTags|    set|spellDamage|targetingArrowText|                text|       type|
+----------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+--------------------+-------+--------------------+------+---------+---------+---------------+---

In [ ]:
"""
Remove samples without artist name, attack and health value
"""

In [7]:
data_authors = data.na.drop(subset=["artist","attack", "health"]).cache() # remember about .cache() operation
data_authors.show(10)

+--------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+---------+--------+--------------------+------+---------+--------------------+--------------------+-------+-----------+---------------+-------------------+--------+--------------------+-----------+-----+---------+--------------+-------+-----------+------------------+--------------------+------+
|        artist|attack|cardClass|classes|collectible|collectionText|cost|dbfId|durability|elite|entourage| faction|              flavor|health|hideStats|           howToEarn|     howToEarnGolden|     id|  mechanics|multiClassGroup|               name|overload|    playRequirements|playerClass| race|   rarity|referencedTags|    set|spellDamage|targetingArrowText|                text|  type|
+--------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+---------+--------+--------------------+------+---------+--------------------+--------------------+-------+-----------+-

In [ ]:
"""
For each column print the number of non Null rows and number of unique rows
"""

'\nFor each column print the number of non Null rows and number of unique rows\n'

In [8]:
for column_name in data_authors.columns:
    rows_number_not_empty = data_authors.filter(col(column_name).isNotNull()).count()
    rows_number_unique = data_authors.select(column_name).distinct().count() # distinct operation may help

    print('{}. Non empty: {}\tUnique values: {}'.format(column_name, rows_number_not_empty, rows_number_unique))

artist. Non empty: 829	Unique values: 234
attack. Non empty: 829	Unique values: 31
cardClass. Non empty: 829	Unique values: 10
classes. Non empty: 9	Unique values: 4
collectible. Non empty: 731	Unique values: 2
collectionText. Non empty: 5	Unique values: 6
cost. Non empty: 829	Unique values: 14
dbfId. Non empty: 829	Unique values: 829
durability. Non empty: 0	Unique values: 1
elite. Non empty: 141	Unique values: 2
entourage. Non empty: 6	Unique values: 7
faction. Non empty: 60	Unique values: 3
flavor. Non empty: 731	Unique values: 732
health. Non empty: 829	Unique values: 30
hideStats. Non empty: 2	Unique values: 2
howToEarn. Non empty: 136	Unique values: 27
howToEarnGolden. Non empty: 181	Unique values: 77
id. Non empty: 829	Unique values: 829
mechanics. Non empty: 553	Unique values: 49
multiClassGroup. Non empty: 9	Unique values: 4
name. Non empty: 829	Unique values: 791
overload. Non empty: 9	Unique values: 4
playRequirements. Non empty: 81	Unique values: 34
playerClass. Non empty: 

In [9]:
data_authors.select('race').distinct().show()

+----------+
|      race|
+----------+
|MECHANICAL|
|      null|
|    MURLOC|
|    DRAGON|
|     TOTEM|
|     BEAST|
|     DEMON|
|    PIRATE|
+----------+



In [10]:
data_authors.select('rarity').distinct().show()

+---------+
|   rarity|
+---------+
|     null|
|     FREE|
|     EPIC|
|     RARE|
|   COMMON|
|LEGENDARY|
+---------+



In [11]:
str2int_rarity = {
    'FREE': 1,
    'COMMON': 2,
    'RARE': 3,
    'LEGENDARY': 4,
    'EPIC': 5,
    'NULL': 0
}

In [ ]:
"""
Using dict above - create a new column with name "rarity_int", where each string will be
converted to an integer

Operation "col" return "pointer" to required column
"""

In [12]:
data_authors = data_authors.withColumn('rarity_int', udf(lambda x: str2int_rarity.get(x, 0), IntegerType())(col('rarity')))

In [ ]:
"""
Calculate average rarity of each race
"""

In [13]:
data_authors.groupby('race').agg(mean('rarity_int').alias('average rarity')).show(10)

+----------+------------------+
|      race|    average rarity|
+----------+------------------+
|MECHANICAL|3.0185185185185186|
|      null|2.6536502546689302|
|    MURLOC| 2.652173913043478|
|    DRAGON|3.2222222222222223|
|     TOTEM|               2.5|
|     BEAST|               2.3|
|     DEMON|2.3333333333333335|
|    PIRATE|2.8947368421052633|
+----------+------------------+



In [14]:
@udf(returnType=IntegerType())
def calculate_number_of_attacks(health, attack):
    if attack == 0:
        return -1
    return health // attack + (0 if health % attack == 0 else 1)

In [ ]:
"""
Using udf defined above, calculate how many attacks needed for each card to kill itself
"""

In [15]:
data_authors_2 = data_authors.withColumn('Attack to kill itself', calculate_number_of_attacks(col("health"), col("attack"))).select('health', 'attack', 'Attack to kill itself')

In [16]:
data_authors_2.show(5)

+------+------+---------------------+
|health|attack|Attack to kill itself|
+------+------+---------------------+
|    10|    10|                    1|
|     5|     3|                    2|
|     3|     2|                    2|
|     2|     2|                    1|
|     4|     5|                    1|
+------+------+---------------------+
only showing top 5 rows



In [ ]:
"""
Here some work on randomizing indexes for dataframe
It is not always easy to assign unique values for each row non-monotonically
"""

In [17]:
random_ids_left = list(range(data_authors.count() + 1))
random_ids_right = list(range(data_authors.count() + 1))

In [18]:
random.shuffle(random_ids_left)
random.shuffle(random_ids_right)

In [ ]:
"""
Create monotonic id
"""

In [19]:
data_authors = data_authors.withColumn('unique_id', monotonically_increasing_id())

In [ ]:
"""
Create left and right indexes for each sample. You will use them for join operation
"""

In [20]:
data_authors = data_authors.withColumn('left_id', udf(lambda x: random_ids_left[x], IntegerType())(col('unique_id')))
data_authors = data_authors.withColumn('right_id', udf(lambda x: random_ids_right[x], IntegerType())(col('unique_id'))).cache()

In [21]:
data_authors.count()

829

In [ ]:
"""
For each sample you need to select another sample, where left_id == right_id and create new row
Resulting dataframe consists of [left_id, name, attack, health, right_id, right_name, right_attack, right_health]
"""

In [ ]:
"""
Small hint: data_authors.select(col('name').alias('right_name'))
"""

In [ ]:
"""
Small hint: you can join table with itself
"""

In [53]:
data_crossed = data_authors.alias("left").join(data_authors.alias("right"),
    col("left.left_id") == col("right.right_id"), "inner")

data_crossed.show(10)

+--------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+---------+--------+--------------------+------+---------+--------------------+--------------------+-------+-----------+---------------+-------------------+--------+--------------------+-----------+-----+---------+--------------+-------+-----------+------------------+--------------------+------+----------+---------+-------+--------+-----------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+---------+-------+--------------------+------+---------+--------------------+--------------------+--------+-----------+---------------+--------------------+--------+----------------+-----------+----------+---------+--------------+-------+-----------+------------------+--------------------+------+----------+---------+-------+--------+
|        artist|attack|cardClass|classes|collectible|collectionText|cost|dbfId|durability|elite|entourage| faction|              flavor|h

In [54]:
data_crossed = data_crossed.select(col('left.left_id'),
                                   col('left.name'), col('left.attack'),
                                   col('left.health'),
                                   col('right.right_id'), col('right.name').alias('right_name'),
                                   col('right.attack').alias('right_attack'), col('right.health').alias('right_health'))

data_crossed.show(10)

+-------+-------------------+------+------+--------+--------------------+------------+------------+
|left_id|               name|attack|health|right_id|          right_name|right_attack|right_health|
+-------+-------------------+------+------+--------+--------------------+------------+------------+
|    567|            Icehowl|    10|    10|     567|    Toxic Sewer Ooze|           4|           3|
|    592|       Wailing Soul|     3|     5|     592|  Goblin Auto-Barber|           3|           2|
|    662|  Sunfury Protector|     2|     3|     662|         Onyx Bishop|           3|           4|
|    160|       Dark Peddler|     2|     2|     160|  Shade of Naxxramas|           2|           2|
|    525|   Genzo, the Shark|     5|     4|     525|  Bilefin Tidehunter|           2|           1|
|    118|   Reliquary Seeker|     1|     1|     118|       Prophet Velen|           7|           7|
|    461|Injured Blademaster|     4|     7|     461|Netherspite Histo...|           1|           3|


In [ ]:
"""
Now calculate how many attacks required for left card to kill right card
"""

In [55]:
data_crossed = data_crossed.withColumn('Attacks to kill', calculate_number_of_attacks(col('right_health'), col('attack')))

In [56]:
data_crossed.filter(col('Attacks to kill') > 10).show(100)

+-------+--------------------+------+------+--------+----------+------------+------------+---------------+
|left_id|                name|attack|health|right_id|right_name|right_attack|right_health|Attacks to kill|
+-------+--------------------+------+------+--------+----------+------------+------------+---------------+
|    344|Arcane Nullifier ...|     2|     5|     344|Jade Golem|          25|          25|             13|
|    804|Netherspite Histo...|     1|     3|     804|Jade Golem|          21|          21|             21|
|    106|             Piranha|     1|     1|     106|     Ysera|           4|          12|             12|
|    298|            Tabbycat|     1|     1|     298|Jade Golem|          12|          12|             12|
|    793|       Pantry Spider|     1|     3|     793|Jade Golem|          26|          26|             26|
+-------+--------------------+------+------+--------+----------+------------+------------+---------------+



In [ ]:
"""
Additional task OPTIONAL

Calculate who will win (left or right), if left one starts first
"""